<a href="https://colab.research.google.com/github/ArpitaChatterjee/Biometric-Attendance/blob/main/Attendance_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip  install dlib==19.18.0

In [2]:
!pip install face_recognition
import dlib
import face_recognition

     |████████████████████████████████| 100.1 MB 12 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=07bf5d244a7f930c9a235e1d5c050ea8ec3066b27707bd611ab3540bc0305d9d
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [3]:
!pip install opencv-python
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

In [4]:
path = '/content/drive/My Drive/Colab Notebooks/Face Recognition/ImagesAttendance'
images=[]
classNames=[]
mylist= os.listdir(path)
print(mylist)


['Bill Gates.jpg', 'Jack Ma.jpg', 'Elon Musk.jpg']


In [5]:
for cl in mylist:
  curImg = cv2.imread(f'{path}/{cl}')
  images.append(curImg)
  classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Bill Gates', 'Jack Ma', 'Elon Musk']


In [6]:
def findEncodings(images):
  encodeList =[]
  for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)
  return encodeList

encodelistknown = findEncodings(images)
print(len(encodelistknown))

3


In [7]:
print('encoding Complete')

encoding Complete


In [8]:
from datetime import datetime

In [9]:
#TO Mark Attendence
def markAttendence(name):
  with open('/content/drive/My Drive/Colab Notebooks/Face Recognition/Attendance.csv', 'r+') as f:
    mydatalist = f.readlines()
    namelist=[]
    for line in mydatalist:
      entry = line.split(',')
      namelist.append(entry[0])
    if name not in namelist:
      now = datetime.now()
      dtstr = now.strftime('%H:%M:%S')
      f.writelines(f'\n{name}, {dtstr}')

markAttendence('Elon')
markAttendence('Bill Gates')